In [1]:
import pyspark
from pyspark.sql import SparkSession

In [2]:
spark = SparkSession.builder \
    .master("local[*]") \
    .appName('test') \
    .getOrCreate()

### GROUP BY

In [3]:
df_green = spark.read.parquet('data/pq/green/*/*')

In [4]:
df_green.createOrReplaceTempView('green')

In [9]:
df_green_revenue = spark.sql("""
SELECT 
    date_trunc('hour', lpep_pickup_datetime) AS hour,
    PULocationID AS zone,

    SUM(total_amount) AS amount,
    COUNT(1) as number_records
FROM
    green
WHERE
    lpep_pickup_datetime >= '2020-01-01 00:00:00'
GROUP BY
    1,2
""")

In [10]:
df_green_revenue.show()

+-------------------+----+------------------+--------------+
|               hour|zone|            amount|number_records|
+-------------------+----+------------------+--------------+
|2020-01-17 11:00:00|  41| 513.6600000000003|            46|
|2020-01-30 13:00:00|  75| 1330.859999999999|            76|
|2020-01-27 20:00:00| 244|398.00000000000006|            16|
|2020-01-16 10:00:00|  74|1021.2199999999987|            80|
|2020-01-24 17:00:00| 260| 308.0600000000001|            27|
|2020-01-03 01:00:00|  80|             80.61|             6|
|2020-01-08 14:00:00|  33| 687.7900000000001|            34|
|2020-01-07 19:00:00|  66|324.78000000000003|            19|
|2020-01-14 12:00:00|  21|             90.22|             2|
|2020-01-17 17:00:00|  41| 845.2499999999994|            66|
|2020-01-02 09:00:00|   7|367.50000000000017|            30|
|2020-01-15 20:00:00| 145|             39.41|             2|
|2020-01-06 19:00:00|  92| 293.2000000000001|            18|
|2020-01-24 11:00:00| 12

In [17]:
df_green_revenue \
    .repartition(20) \
    .write.parquet('data/report/revenue/green', mode='overwrite')

In [12]:
df_yellow = spark.read.parquet('data/pq/yellow/*/*')

In [13]:
df_yellow.createOrReplaceTempView('yellow')

In [14]:
df_yellow_revenue = spark.sql("""
SELECT 
    date_trunc('hour', tpep_pickup_datetime) AS hour,
    PULocationID AS zone,

    SUM(total_amount) AS amount,
    COUNT(1) as number_records
FROM
    yellow
WHERE
    tpep_pickup_datetime >= '2020-01-01 00:00:00'
GROUP BY
    1,2
""")

In [16]:
df_yellow_revenue.show()

+-------------------+----+------------------+--------------+
|               hour|zone|            amount|number_records|
+-------------------+----+------------------+--------------+
|2020-01-01 08:00:00| 162| 767.8100000000003|            46|
|2020-01-04 18:00:00| 140|2618.9099999999985|           169|
|2020-01-27 16:00:00|  88|1478.9999999999995|            53|
|2020-01-05 02:00:00| 114| 5647.420000000002|           329|
|2020-01-19 23:00:00|  90|2217.8799999999987|           150|
|2020-01-23 17:00:00| 140| 6545.520000000008|           364|
|2020-01-09 11:00:00| 107|3677.2599999999966|           240|
|2020-01-26 20:00:00| 239|2532.9399999999987|           178|
|2020-01-03 23:00:00| 132| 24519.23999999997|           448|
|2020-01-26 20:00:00| 236|3540.5199999999977|           242|
|2020-01-24 19:00:00| 186|11532.320000000029|           625|
|2020-01-24 03:00:00| 264|             249.6|            11|
|2020-01-31 15:00:00| 143|2763.2299999999977|           182|
|2020-01-23 21:00:00| 14

In [18]:
df_yellow_revenue \
    .repartition(20) \
    .write.parquet('data/report/revenue/yellow', mode='overwrite')

### JOIN

In [19]:
df_green_revenue = spark.read.parquet('data/report/revenue/green/')
df_yellow_revenue = spark.read.parquet('data/report/revenue/yellow/')

In [20]:
df_green_revenue_temp = df_green_revenue \
    .withColumnRenamed('amount', 'green_amount') \
    .withColumnRenamed('number_records', 'green_number_records') 

df_yellow_revenue_temp = df_yellow_revenue \
    .withColumnRenamed('amount', 'yellow_amount') \
    .withColumnRenamed('number_records', 'yellow_number_records') 

In [22]:
df_join = df_green_revenue_temp.join(df_yellow_revenue_temp, on=['hour', 'zone'], how='outer')

In [23]:
df_join.show()

+-------------------+----+-----------------+--------------------+------------------+---------------------+
|               hour|zone|     green_amount|green_number_records|     yellow_amount|yellow_number_records|
+-------------------+----+-----------------+--------------------+------------------+---------------------+
|2020-01-01 00:00:00|   7|769.7299999999997|                  45| 455.1700000000001|                   38|
|2020-01-01 00:00:00|  13|             NULL|                NULL|1214.8000000000002|                   56|
|2020-01-01 00:00:00|  18|              7.8|                   1|               5.8|                    1|
|2020-01-01 00:00:00|  32|68.94999999999999|                   2|              18.0|                    1|
|2020-01-01 00:00:00|  34|             NULL|                NULL|              19.3|                    1|
|2020-01-01 00:00:00|  83|94.09999999999998|                   7|               9.8|                    1|
|2020-01-01 00:00:00|  90|           

In [24]:
df_join.write.parquet('data/report/revenue/total', mode='overwrite')

In [27]:
df_join.columns

['hour',
 'zone',
 'green_amount',
 'green_number_records',
 'yellow_amount',
 'yellow_number_records']

In [29]:
df_zones = spark.read.parquet('zones/')

In [30]:
df_result = df_join.join(df_zones, df_join.zone == df_zones.LocationID)

In [34]:
df_result.drop('LocationID', 'zone').write.parquet('tmp/revenue-zones')